In [3]:
# import the wrapper
from p21cmfastwrapper import *

TypeError: module() takes at most 2 arguments (3 given)

In [7]:
# The wrapper uses a parameter file to initialize the simulation for better overview
# One can initialize the class by
sim = Simulation(save_ondisk=False, write_cache=False, save_inclass=True, debug=True)
# important arguments for initializing the Simulation class are:
'''save_inclass (bool): If set true, results are saved as a list in the class, very useful for testing and quick analysis. If False, results are saved as a file
save_ondisk (bool): If set True, save results on disk
write_cache (bool): If true, use the included 21cmfast cache, useful when doing repeated simulations with same seed and init conditions.
data_path (str): path for saving the results of save_ondisk is True
file_name (str): filename for the runs, final name will be: filename + run_id + .h5
override (bool): If True, old runs will be overridden
debug (bool): Print many things along the way if something is screwed again.'''

use astro_params default config: True
use cosmo_params default config: True
use user_params default config: False
use flag_options default config: False
use global_params default config: True
./data/ run_* 1 ['run_0']
Using 21cmFAST version 0.1.dev1836+gf7c01bf


'save_inclass (bool): If set true, results are saved as a list in the class, very useful for testing and quick analysis. If False, results are saved as a file\nsave_ondisk (bool): If set True, save results on disk\nwrite_cache (bool): If true, use the included 21cmfast cache, useful when doing repeated simulations with same seed and init conditions.\ndata_path (str): path for saving the results of save_ondisk is True\nfile_name (str): filename for the runs, final name will be: filename + run_id + .h5\noverride (bool): If True, old runs will be overridden\ndebug (bool): Print many things along the way if something is screwed again.'

In [3]:
# To just run a single box using the parameters one can just 
sim.run_box()

In [8]:
# To simulate a lightcone, one can do
sim.run_cone()
# Now we clear the cached simulations for the next step
#sim.clear()

In [ ]:
# All parameters can be changed on the fly without re-initializing the class, e.g.
params = {"redshift": 8, "max_redshift": 12, "random_seed": 1, "astro_params": {"HII_EFF_FACTOR": 29}} 
# Note: It is important to preserve the dict structure defined in parameter.yaml for in-situ changes in parameters
# This dict needs to be passed as an argument to the wrapper
sim.run_lightcone(kargs=params)

In [ ]:
# One can also do multiple runs with different parameters, e.g. we want two realizations of two HII_EFF_FACTORs
params = {"random_seed": [1,2], "astro_params": {"HII_EFF_FACTOR": [29,31]}}
sim.run_fixed_multi_lightcone(rargs=params)
# The result will be four simulations stored in the class

In [ ]:
# One can sample from a distribution (callable function) given by samplef and a dict of ranges
# This is useful to create a database
# For cluster deployment, change the threads argument to the number of available cpu cores
samplef = (lambda a,b: np.random.uniform(a,b))
args = {"astro_params": 
            {"NU_X_THRESH": [400,700], 
            "HII_EFF_FACTOR": [20,40],
            "L_X": [30,50],
            "ION_Tvir_MIN": [4,5]},
        "cosmo_params":
            {"OMm": [0.1,0.3]},
        "global_params": 
            {"M_WDM": [1,3]}}
sim.run_samplef(nruns=12, args=args, samplef=samplef, threads = 6)

In [ ]:
# Now we visualize the result
# Visualize the power spectrum computed in 6 bins
sim.plot_ps(bins=6,compare=True)
# Plot the global quantities of the simulations
sim.plot_global_properties(observational_axis=True)
# Plot various fields of the simulation
sim.plot_imshow(fields=["brightness_temp","xH_box", "density"])

In [ ]:
# If a function is unclear, one can always look at the docstring for help
sim.plot_ps?

In [ ]:
np.array([1,23,np.NaN]).max()

In [ ]:
import numpy as np

In [ ]:
test = True
while test:
    print("hi :3")
    test = False

In [12]:
import py21cmfast as p21c
user_params = p21c.UserParams(
    HII_DIM=70, BOX_LEN=200, N_THREADS = 6
)
astro_params = p21c.AstroParams(
    INHOMO_RECO = True, USE_TS_FLUCT = True
)
lightcone = p21c.run_lightcone(
    global_quantities=["brightness_temp"],
    direc='_cache',
    write=False,
    user_params=user_params,
    astro_params=astro_params,
    redshift=5.5,
    max_redshift=20
)

/home/potato/Documents/projects/master/p21cm-venv/lib/python3.11/site-packages/py21cmfast/_utils.py:412: UserWarning: The following parameters to AstroParams are not supported: ['USE_TS_FLUCT']
  warnings.warn(
/home/potato/Documents/projects/master/p21cm-venv/lib/python3.11/site-packages/py21cmfast/inputs.py:515: UserWarning: The USE_INTERPOLATION_TABLES setting has changed in v3.1.2 to be default True. You can likely ignore this warning, but if you relied onhaving USE_INTERPOLATION_TABLES=False by *default*, please set it explicitly. To silence this warning, set it explicitly to True. Thiswarning will be removed in v4.
  warnings.warn(


In [4]:
from powerbox import get_power
import numpy as np
from matplotlib import pyplot as plt
import py21cmfast as p21c

/home/potato/Documents/projects/master/p21cm-venv/lib/python3.11/site-packages/py21cmfast/_cfg.py:58: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/home/potato/Documents/projects/master/p21cm-venv/lib/python3.11/site-packages/py21cmfast/_cfg.py:42: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


In [14]:
lc = p21c.outputs.LightCone.read("../data/res_comp/110_res_False_.h5")
# lc.lightcone_redshifts[800:850]
lc.cell_size

1.8181818181818181

In [6]:
def ps2d(data, size):
    #size = data.shape
    ps_perp,k_perp, _ = get_power(data, boxlength=size, res_ndim=2, bins = 20, 
                                  ignore_zero_mode=True, bin_ave=True) 
    ps_par, k_par, _ = get_power(data.T, boxlength=size, res_ndim=1, bins = 15, 
                                 ignore_zero_mode=True, bin_ave=True)
    ps_perp = np.mean(ps_perp,axis=1)
    ps_par = np.mean(ps_par, axis=(1,2))
    v = np.prod(size)
    ps = np.outer(ps_perp*k_perp**2, ps_par*k_par)/(2*np.pi**2*v)
    return k_perp, k_par, ps

In [7]:
k_perp, k_par, ps = ps2d(lc.brightness_temp[:,:,800:850],
                         lc.cell_size*np.array([*lc.brightness_temp.shape[:2], 50]))

In [3]:
plt.imshow(ps)
kx = np.round(k_par)
#plt.xticks(np.linspace(0,14,14), k_par)
#plt.yticks(np.linspace(0,14,14), k_perp)

NameError: name 'plt' is not defined

In [9]:
k_par

array([0.03141593, 0.07853982, 0.12566371, 0.1727876 , 0.23561945,
       0.28274334, 0.32986723, 0.39269908, 0.43982297, 0.48694686,
       0.54977871, 0.5969026 , 0.64402649, 0.70685835, 0.75398224])

In [16]:
print(lc.brightness_temp.shape)
lcr = lc.lightcone_redshifts
print(lcr.min(), lcr.max(), lcr.shape)

(110, 110, 1967)
5.500000006120298 35.03091305129813 (1967,)


In [13]:
lc.brightness_temp = lc.brightness_temp[:,:,:1000]

(1,)